In [2]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [55]:
#Use the Notebook to build the code to scrape the following Wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
from bs4 import BeautifulSoup as bs
html = requests.get(url).text
soup =bs(html,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'})
headers= My_table.find_all('th')

Cols=[]
for header in headers :
    Cols.append(header.string.strip())
Cols

['Postcode', 'Borough', 'Neighbourhood']

In [63]:
# 3. To create the above dataframe:
my_data=[]
rows=My_table.find_all('tr')[1:]
for row in rows:
    vals=row.find_all('td')
    vals=[x.text.strip() for x in vals]
    my_data.append(vals)
df = pd.DataFrame(my_data, columns=Cols)
df.head(10)


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [67]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighbourhood'] = np.where(df['Neighbourhood']== 'Not assigned', df['Borough'],df['Neighbourhood'])

In [68]:
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Queen's Park


In [69]:
df.shape

(287, 3)